In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [16]:
k_list = [2, 3, 4, 5, *range(10, 101, 10)]
n = 100
q = 100
p_list = [0.5, 1, 2, 5, None] # None instead of inf to calculate Chebyshev distance with Lp_norm()

In [17]:
def gen_data(q, n, k):
  pts = np.random.uniform(0, 1, size=(q, n, k))
  og = np.zeros(k)
  for i in range(0, q):
    for j in range(0, n):
      while np.allclose(pts[i][j], og):
        pts[i][j] = np.random.uniform(0, 1, size=k)
  return pts

def Lp_norm(p, a, b):
  if (p == None): # Chebyshev distance
    return max(abs(a-b))
  return (np.sum((abs(a - b))**p))**(1/p)

def distance(p, pts, q, n, k): # Lp norm for pts of size (q, n, k)
  og = np.zeros(k)
  dists = np.empty((q, n))
  for i in range(0, q):
    for j in range(0, n):
      dists[i][j] = Lp_norm(p, pts[i][j], og)
  return dists

def distances(p_list, pts, q, n, k): # Lp norms for all p in p_list for pts of size (q, n, k)
  dists = {}
  for p in p_list:
    dist = distance(p, pts, q, n, k)
    stats = {}
    stats["Dmin"] = np.min(dist, axis=1)
    stats["Min_avg"] = np.mean(stats["Dmin"])

    stats["Dmax"] = np.max(dist, axis=1)
    stats["Max_avg"] = np.mean(stats["Dmax"])

    stats["Dmean"] = np.mean(dist, axis=1)
    stats["Avg_avg"] = np.mean(stats["Dmean"])

    stats["Dvar"] = np.var(dist, axis=1, ddof=1)
    stats["Var_avg"] = np.mean(stats["Dvar"])

    stats["Ctr"] = np.divide(stats["Dmax"] - stats["Dmin"], stats["Dmin"])
    stats["Ctr_avg"] = np.mean(stats["Ctr"])

    finalobj = {}
    finalobj["dist"] = dist
    finalobj["stats"] = stats

    dists[p] = finalobj
  return dists

In [18]:
data = {}

for k in k_list:
  data[k] = gen_data(q, n, k)

dists_and_stats = {}
for k in k_list:
  dists_and_stats[k] = distances(p_list, data[k], q, n, k)

dists_and_stats

{2: {0.5: {'dist': array([[2.87062998, 1.42659018, 3.63190864, ..., 1.24282556, 1.86599413,
           2.38979122],
          [2.70115902, 1.39642124, 2.06081871, ..., 1.58362201, 1.47118938,
           3.7753752 ],
          [2.43735448, 1.73113788, 2.29559997, ..., 1.5187445 , 2.82780405,
           2.65023403],
          ...,
          [0.51274401, 1.57926826, 0.93839981, ..., 0.89315812, 0.85414389,
           0.59884734],
          [1.31026005, 2.64342224, 3.8559555 , ..., 0.54537014, 1.00946122,
           2.89788805],
          [1.00913286, 2.335415  , 1.92222048, ..., 0.98937623, 0.89093296,
           2.75416555]]),
   'stats': {'Dmin': array([0.21676839, 0.48096308, 0.47336594, 0.14028632, 0.10701482,
           0.03496852, 0.21196866, 0.24349112, 0.26310371, 0.27909613,
           0.0099559 , 0.37758089, 0.28689788, 0.26617158, 0.09643549,
           0.18368727, 0.26703868, 0.1631125 , 0.11211195, 0.21351822,
           0.39788865, 0.27477996, 0.67467895, 0.2310669 , 0.18343